# Euchre
Goal: code up a playable euchre game to better understand winning probabilities of different starting hands  
Given: Right, Ace, Queen, off Ace, off ten; 3-suited, dealer spot - what's the likelihood of winning each number of tricks that round?  
Genetic Algorithm to determine next play would be cool - start with random strategy, adapt to new weights as games progress

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import sys

sys.path.insert(0, 'C:/Users/jerem/Desktop/nonsense/euchre/')

In [2]:
from utils import EuchreGame

In [123]:
euchre_game = EuchreGame()

In [124]:
euchre_game.score

[0, 0]

In [125]:
euchre_game.dealer

'p1'

In [126]:
euchre_game.next_to_deal

['p2', 'p3', 'p4', 'p1']

In [127]:
euchre_game.print_score()

Current score: 0-0


In [128]:
euchre_game.card_suits

['S', 'C', 'H', 'D']

In [129]:
euchre_game.card_values

['9', 'T', 'J', 'Q', 'K', 'A']

In [130]:
deck_of_cards = euchre_game.shuffle_deck_of_cards()
for card in deck_of_cards:
    print(card, end=',')

9_S,9_C,9_H,9_D,T_S,T_C,T_H,T_D,J_S,J_C,J_H,J_D,Q_S,Q_C,Q_H,Q_D,K_S,K_C,K_H,K_D,A_S,A_C,A_H,A_D,

In [131]:
player_hands, card_flipped_up = euchre_game.deal_hand()

In [132]:
player_hands

{'p1': ['K_H', 'A_C', 'T_H', 'Q_D', 'A_H'],
 'p2': ['Q_H', 'J_H', '9_S', 'T_S', 'A_D'],
 'p3': ['K_D', 'Q_C', 'A_S', 'J_S', '9_D'],
 'p4': ['K_C', '9_H', '9_C', 'T_C', 'T_D']}

In [133]:
card_flipped_up

'J_C'

In [134]:
def eval_flipped_card(suit, hand) -> bool:
    """
    Function to evaluate if a player will order up trump
    If hand has at least 3 trump cards
    
    Returns True/False
    """
    trumps = 0
    for card in hand:
        if card[-1] == suit:
            trumps += 1
    if trumps >= 3:
        return True
    else:
        False

In [135]:
if eval_flipped_card(suit='S', hand=['K_S', 'A_H', 'A_C', 'J_C', 'J_S']):
    print('order up trump')

In [136]:
if eval_flipped_card(suit='S', hand=['K_S', 'A_H', 'A_S', 'J_C', 'J_S']):
    print('order up trump')

order up trump


In [137]:
def choose_open_trump(hand, card_flipped_up) -> str:
    """
    Function to choose trump after card is turned down
    If hand has at least 3 trump cards
    
    Returns True/False
    """
    card_suits = ['S', 'C', 'H', 'D']
    card_suits.remove(card_flipped_up[-1])
    for suit in card_suits:
        trumps = 0
        for card in hand:
            if card[-1] == suit:
                trumps += 1
        if trumps >= 3:
            return suit
    else:
        return None

In [138]:
choose_open_trump(hand=['K_S', 'A_S', 'A_C', 'J_C', 'J_S'], 
                  card_flipped_up='A_D')

'S'

In [139]:
choose_open_trump(hand=['K_S', 'A_H', 'A_C', 'J_C', 'J_S'], 
                  card_flipped_up='A_D')

In [140]:
calling_player, trump = euchre_game.determine_trump(card_flipped_up=card_flipped_up,
                           player_hands=player_hands)

In [141]:
player_hands, card_flipped_up = euchre_game.deal_hand()
calling_player, trump = euchre_game.determine_trump(card_flipped_up=card_flipped_up,
                           player_hands=player_hands)

In [142]:
print(player_hands)
print(calling_player)
print(trump)

{'p1': ['T_C', 'A_D', 'K_S', '9_D', 'J_C'], 'p2': ['9_H', 'T_D', 'Q_C', 'A_S', 'Q_D'], 'p3': ['J_S', 'J_D', 'Q_H', 'A_H', 'A_C'], 'p4': ['9_C', 'J_H', 'K_C', 'Q_S', 'T_H']}
None
None


In [148]:
def play_card(hand, 
              trump,
              cards_in_play=[], 
              suit_led=None):
    """
    Function to return value of card to play in hand
    TODO: check if partner has winning card_in_pot
    """
    card_values = {
        '9':1, 
        'T':2, 
        'J':3, 
        'Q':4, 
        'K':5, 
        'A':6
    }
    # play last card
    if len(hand)==1:
        return hand[0]
    # lead card
    # TODO: develop logic for what card to lead
    if len(cards_in_play)<1:
        return hand[0]
    # follow suit
    if suit_led is not None:
        # play highest card in the suit played
        card_to_play_points = -1
        idx_to_return = -1
        for idx,card in enumerate(hand):
            if card[-1]==suit_led:
                card_points = card_values[card[0]]
                if card_points > card_to_play_points:
                    card_to_play_points = card_points
                    idx_to_return = idx
        return hand[idx_to_return]
    # play other highest non-trump card
    else:
        card_to_play_points = -1
        idx_to_return = -1
        for idx,card in enumerate(hand):
            if card[-1]!=trump:
                card_points = card_values[card[0]]
                if card_points > card_to_play_points:
                    card_to_play_points = card_points
                    idx_to_return = idx
        if idx_to_return > -1:
            return hand[idx_to_return]
        else:  # only has trump left, play lowest trump
            trump_card_points = 9
            for idx,card in enumerate(hand):
                card_points = card_values[card[0]]
                if card_points < trump_card_points:
                    trump_card_points = card_points
                    idx_to_return = idx
            return hand[idx_to_return]

In [149]:
play_card(hand=['J_H', 'A_D', '9_H', 'K_S', 'J_D'], 
          trump='S',
          cards_in_play=['K_H', 'T_H'], 
          suit_led='H')

'J_H'

In [150]:
def play_trick(player_hands, 
               trump, 
               next_to_play_list):
    """
    Function to play one full trick
    
    """
    cards_in_play = {}
    suit_led = None
    for idx, player in enumerate(next_to_play_list):
        card_to_play = play_card(hand=player_hands[player], 
                                 trump=trump,
                                 cards_in_play=cards_in_play, 
                                 suit_led=suit_led)
        print(player_hands[player])
        # add card_to_play
        cards_in_play[player] = card_to_play
        # update player hands after player has played card
        player_hands[player].remove(card_to_play)
        print(f'Player {player} plays {card_to_play}')
        if idx==0:
            suit_led = card_to_play[-1]
            player_led = player
            print(f'Suit led: {suit_led}')
    return cards_in_play, player_led

In [151]:
cards_in_play, player_led = play_trick(player_hands=player_hands, 
                                       trump=trump, 
                                       next_to_play_list=euchre_game.next_to_deal)

['T_D', 'Q_C', 'A_S', 'Q_D']
Player p2 plays T_D
Suit led: D
['J_S', 'J_D', 'Q_H', 'A_C']
Player p3 plays J_D
['9_C', 'J_H', 'K_C', 'Q_S', 'T_H']
Player p4 plays T_H
['T_C', 'A_D', 'K_S', '9_D', 'J_C']
Player p1 plays A_D


In [152]:
cards_in_play

{'p2': 'T_D', 'p3': 'J_D', 'p4': 'T_H', 'p1': 'A_D'}

In [153]:
player_led

'p2'

In [154]:
def determine_trick_winner(cards_in_play, 
                           trump, 
                           player_led) -> str:
    """
    Determine winner of trick
    
    Returns player that won trick
    """
    return player_led

In [155]:
determine_trick_winner(cards_in_play=cards_in_play, 
                           trump=trump, 
                           player_led=player_led)

'p2'

In [156]:
euchre_game.next_to_deal

['p2', 'p3', 'p4', 'p1']

In [162]:
def play_hand(next_to_play_list):
    """
    Function to play a single hand
    """
    # deal cards
    player_hands, card_flipped_up = euchre_game.deal_hand()
    # choose trump
    calling_player, trump = euchre_game.determine_trump(card_flipped_up=card_flipped_up,
                                                        player_hands=player_hands)

    if trump is not None:
        trick_winners = {p:0 for p in euchre_game.next_to_deal}
        print(f'Trump: {trump}')
        # play trick
        for trick in range(5):
            cards_in_play, player_led = play_trick(player_hands=player_hands, 
                                               trump=trump, 
                                               next_to_play_list=next_to_play_list)
            trick_winner = determine_trick_winner(cards_in_play=cards_in_play, 
                                       trump=trump, 
                                       player_led=player_led)
            trick_winners[trick_winner] += 1
            # update next_to_play_list based on trick_winner
        return trick_winners

    # update score
        # if calling team takes >= 3 tricks = 1 point, if calling team takes 5 tricks = 2 points
        # if non-calling team takes >=3 tricks = 2 points
    else:
        print('Trump not found')
        euchre_game.dealer = euchre_game.next_to_deal.pop(0)
        # reset self.next_to_deal order for next turn
        euchre_game.next_to_deal = euchre_game.next_to_deal.append(euchre_game.dealer)

In [164]:
next_to_play_list = euchre_game.next_to_deal
trick_winners = play_hand(next_to_play_list=next_to_play_list)

Trump: D
['Q_D', 'Q_S', 'J_S', 'J_D', 'T_C']
Player p2 plays Q_D
Suit led: D
['9_C', 'J_C', '9_H', 'A_C', 'K_S']
Player p3 plays K_S
['A_S', 'K_C', '9_S', 'A_H', 'J_H']
Player p4 plays J_H
['Q_C', 'A_D', 'Q_H', 'K_D', 'T_D']
Player p1 plays A_D
['Q_S', 'J_S', 'J_D', 'T_C']
Player p2 plays Q_S
Suit led: S
['9_C', 'J_C', '9_H', 'A_C']
Player p3 plays A_C
['A_S', 'K_C', '9_S', 'A_H']
Player p4 plays A_S
['Q_C', 'Q_H', 'K_D', 'T_D']
Player p1 plays T_D
['J_S', 'J_D', 'T_C']
Player p2 plays J_S
Suit led: S
['9_C', 'J_C', '9_H']
Player p3 plays 9_H
['K_C', '9_S', 'A_H']
Player p4 plays 9_S
['Q_C', 'Q_H', 'K_D']
Player p1 plays K_D
['J_D', 'T_C']
Player p2 plays J_D
Suit led: D
['9_C', 'J_C']
Player p3 plays J_C
['K_C', 'A_H']
Player p4 plays A_H
['Q_C', 'Q_H']
Player p1 plays Q_H
['T_C']
Player p2 plays T_C
Suit led: C
['9_C']
Player p3 plays 9_C
['K_C']
Player p4 plays K_C
['Q_C']
Player p1 plays Q_C


In [165]:
euchre_game.next_to_deal

['p2', 'p3', 'p4', 'p1']

In [166]:
euchre_game.dealer

'p1'

In [167]:
trick_winners

{'p2': 5, 'p3': 0, 'p4': 0, 'p1': 0}

In [ ]:
# if calling team takes >= 3 tricks = 1 point, if calling team takes 5 tricks = 2 points
# if non-calling team takes >=3 tricks = 2 points

def calc_hand_score(trick_winners, 
                    calling_player):
    """
    Calculate score for one hand given trick_winners and calling_player
    """
    if calling_player in ['p1','p3']:
        calling_team = 't1'
    

In [ ]:
def play_full_game(self):
    """
    Play full game
    """
    # TODO: pick random dealer to start game
    next_to_play_list = euchre_game.next_to_deal
    
    # while score < 10:
    # self.play_hand()
    euchre_game.play_hand(next_to_play_list)
    trick_winners = play_hand(next_to_play_list=next_to_play_list)
    # update score
    

In [ ]:
# players
# teams
# players hand

In [ ]:
# actions: 
# deal new hand
# choose trump
# play card
# check if team wins
# keep track of dealer - get next dealer

# left bauer switching to trump
    # keep track of trump each hand
# loners